<a href="https://colab.research.google.com/github/koojahyeob/ToBigs20/blob/main/1_Random_Forest_Example_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### matplotlib 한글 폰트 깨짐 방지
### 이 셀 실행시키고 런타임->런타임 다시 시작 한 다음에
### 이 셀은 건너뛰고 아래부터 실행 시키면 됨.

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/opentype: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/opentype/font-awesome: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/opentype/mathjax: caching, new cache contents: 24 fonts, 0 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 5 dirs
/usr/share/fonts/truetype/font-awesome: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/glyphicons: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 di

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd #필요한 라이브러리 import
import numpy as np #필요한 라이브러리 import
import matplotlib.pyplot as plt #필요한 라이브러리 import
from matplotlib import font_manager, rc # matplot 한글 폰트 깨짐을 방지하기 위한
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') # 폰트 "맑은 고딕" 설정
import warnings
warnings.filterwarnings(action='ignore') # 경고 무시


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI/bundang.csv") # df에 csv 파일 저장
df = df.drop(columns=['시군구','단지명','건축년도']) # 컬럼 삭제
df.head() # 데이터 처음 5줄 출력

FileNotFoundError: ignored

In [ ]:
df.isna().sum() # 결측치 확인

In [ ]:
plt.hist(df['거래금액'],bins=100) # 거리 금액에 대한 히스토그램 파악 // 막대 100개 설정

In [ ]:
df['거래금액_log'] = np.log(df['거래금액']+1) # 로그 변환
df = df.drop(columns=['거래금액']) # 로그 변환한 컬럼이 생겼으니 기존 컬럼은 삭제
plt.hist(df['거래금액_log'],bins=100) # 로그 변환한 거리 금액에 대한 히스토그램 파악 // 막대 100개 설정

In [ ]:
from sklearn.model_selection import train_test_split # sklearn 라이브러리
from sklearn.ensemble import RandomForestRegressor # 앙상블 모델 중 랜덤포레스트 import

Y = df['거래금액_log'].values # Y 값 설정
Y = Y.astype('float32') # Y 값 문자열 타입 설정
X = df.drop(columns=['거래금액_log'])  # X 값에서 거래금액_log 값 제거
X = X.astype('float32') # X 값 문자열 타입 설정

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42) # 학습 데이터 테스트 데이터 8:2로 나누기
RF_model = RandomForestRegressor(n_estimators=2000, random_state=42)
# n_estimators : boosting 반복횟수 설정
# random_state : 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값
RF_model.fit(X_train,Y_train) # 트레이닝 데이터 학습시키기

NameError: ignored

In [ ]:
Train_Prediction = (RF_model.predict(X_train)) # 트레이닝 데이터로 예측
Test_Prediction = (RF_model.predict(X_test)) # 학습된 모델을 통해 예측

RF_Train_MAE = round(np.mean(np.abs(Y_train-Train_Prediction)),3) # train data에 대한 평균절대오차 값 설정 (3자리까지 반올림)
RF_Train_MAPE = round((np.mean(np.abs((Y_train-Train_Prediction)/(Y_train)))*100),3) # train data에 대한 평균절대 백분율 오차 값 설정 (3자리까지 반올림)

RF_Test_MAE = round(np.mean(np.abs(Y_test-Test_Prediction)),3) # test data에 대한 평균절대오차 값 설정 (3자리까지 반올림)
RF_Test_MAPE = round((np.mean(np.abs((Y_test-Test_Prediction)/(Y_test)))*100),3) # test data에 대한 평균절대 백분율 오차 값 설정 (3자리까지 반올림)

# 위에서 구한 각 값에 대해서 print로 출력
print("Mean Absolute Error for Train Set is : {:.3f}".format(RF_Train_MAE))
print("Mean Absolute Percent Error for Train Set is : {:.3f} %".format(RF_Train_MAPE))
print("Mean Absolute Error for Test Set is : {:.3f}".format(RF_Test_MAE))
print("Mean Absolute Percent Error for Test Set is : {:.3f} %".format(RF_Test_MAPE))

In [ ]:
Original_Predictions = (np.round(np.exp(Test_Prediction+1),-2)).astype(int) #로그 변환한 값 다시 exp 자연상수를 취함으로써 원래 예측값으로 되돌리는 설정 (2자리까지 반올림 // 타입 형태 int)
print(Original_Predictions[:10]) #처음 10개 예측값 출력
Original_Values = (np.round(np.exp(Y_test+1),-2)).astype(int) # 위와 같이 로그 변환감 원래 값으로 되돌림
print(Original_Values[:10]) # 처음 10개 예측값 출력

# 데이터와 컬럼 이름으로 dataframe 생성
data = {'Original':  Original_Values,'Prediction': Original_Predictions}
final = pd.DataFrame(data)
final

In [ ]:
# 중요도 낮은 특성부터 오름차순으로 정렬한 인덱스 생성
sorted_idx = RF_model.feature_importances_.argsort() # argsort() : 중요도 낮은 순서대로 인덱스 정렬
# 중요도 낮은 특성부터 막대그래프 그리기
plt.barh(X.columns[sorted_idx], RF_model.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance") # x축 레이블 생성

# XGBoost

In [ ]:
import xgboost # XGBoost 라이브러리 import

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=41) # X, Y 8:2로 트레이닝 데이터 테스트 데이터 구분
XGB_model = xgboost.XGBRegressor(n_estimators=2000, random_state=41) # boosting 반복횟수 2000으로 설정
XGB_model.fit(X_train,Y_train)

In [ ]:
# 트레이닝 데이터 학습시키기
Train_Prediction = (XGB_model.predict(X_train))
Test_Prediction = (XGB_model.predict(X_test))

XGB_Train_MAE = round(np.mean(np.abs(Y_train-Train_Prediction)),3) # train data에 대한 평균절대오차 값 설정 (3자리까지 반올림)
XGB_Train_MAPE = round((np.mean(np.abs((Y_train-Train_Prediction)/(Y_train)))*100),3) # train data에 대한 평균절대 백분율 오차 값 설정 (3자리까지 반올림)


XGB_Test_MAE = round(np.mean(np.abs(Y_test-Test_Prediction)),3) # test data에 대한 평균절대오차 값 설정 (3자리까지 반올림)
XGB_Test_MAPE = round((np.mean(np.abs((Y_test-Test_Prediction)/(Y_test)))*100),3) # test data에 대한 평균절대 백분율 오차 값 설정 (3자리까지 반올림)

# 위에서 구한 각 값에 대해서 print로 출력
print("Mean Absolute Error for Train Set is : {:.3f}".format(XGB_Train_MAE))
print("Mean Absolute Percent Error for Train Set is : {:.3f} %".format(XGB_Train_MAPE))
print("Mean Absolute Error for Test Set is : {:.3f}".format(XGB_Test_MAE))
print("Mean Absolute Percent Error for Test Set is : {:.3f} %".format(XGB_Test_MAPE))

In [ ]:
xgboost.plot_importance(XGB_model)